In [7]:
import pandas as pd
    
articleFile = '../data/article_full.pkl'       
commentFile="../data/comment_full.pkl"
tableFile="../data/table.xlsx"
cellFile="../data/cell.xlsx"

df_article = pd.read_pickle(articleFile)
df_comment = pd.read_pickle(commentFile)
df_table = pd.read_excel(tableFile,index_col=0)
df_cell =  pd.read_excel(cellFile,index_col=0)
# df_comment_tag = table2tag(df_comment, df_table,df_cell)
# genHTML(art_id,df_article,df_comment_tag)

In [5]:
exportJSON(df_article, df_comment_tag, './search/wmyblog.json')

In [46]:
df_latex = df_comment_tag.loc[df_comment_tag.tag.str.contains('1000')].copy()
df_latex = df_latex.drop_duplicates(subset='reply')
content = """
\\documentclass[twocolumn]{ctexart}
\\usepackage{xeCJK}
\\usepackage{titlesec}
\\usepackage[colorlinks=true, urlcolor=blue, linkcolor=red]{hyperref}
\\usepackage{geometry}
\\setCJKmainfont{SimSun}
\\titleformat*{\\section}{\\Large\\bfseries }
\\titleformat*{\\subsection}{\\centering}
\\geometry{a4paper, scale=0.85}
\\begin{document}

\\title{问答1000}
\\author{王孟源}
\\date{2022-03-11}

\\twocolumn[
\\begin{@twocolumnfalse}
\\maketitle % need full-width title
\\end{@twocolumnfalse}
]
"""
for id in df_latex.id.drop_duplicates():
    title = df_article.loc[df_article.id == id].title.iloc[0]
    content += f'\\section*{{{title}}}\n'
    for idx in df_latex.loc[df_latex.id == id].index:
        date = df_latex.loc[idx,'comment_date'].strftime('%Y-%m-%d %H:%M')
        reply = df_latex.loc[idx,'reply'].replace('<br>','\n').replace('%',' \% ').replace('$',' \$ ').replace('_','\_').replace('&','\&').replace('#','\#').replace('^','\^').replace('<br/>','\n')
        content += f'\\subsection*{{{date}}}\n'
        content += reply

content += '\n \\end{document}'
with open('1000.tex','w',encoding='utf8') as f:
    f.write(content)

In [82]:
import re

rep_list = [
    ('&lt;','<'), ('&gt;','>'), ('&amp;','&'),
    ('<p>','\n'), ('</p>','\n'), ('<br>','\n'),('<br/>','\n'),
    ('%','\%'), ('$','\$'), ('&','\&'),('#','\#'), ('^','\^'), ('_','\_'), ('===','='),
    ('<strike>','\\cancel{'), ('</strike>','}'),
    #math
    ('σ','$\sigma$'), ('r\^2','$r^2$'), ('λ','$\lambda$'), ('π','$\pi$'), ('r\^4','$r^4$'),
    ('∝','$\propto$'), ('ρ','$\\rho$'), ('≈','$\\approx$'), ('\^','\^{}'),
    #special
    ('5d9478403b1a5b12c09b10aaadcdcdcb','empty')
]
rem_list = [
    '\r\n','p="">','<em>','</em>',
    '</span>','<div>','</div>','div>','<h2','h2>',
    '<strong>','</strong>','<code>','</code>','</font>','font color="#656565" face="Microsoft YaHei, Arial">',
    '<div class="articlebox">', '<div id="article\_show\_content">',
    '<div class="POST" style="font-size:16px">', '<p class="DATE">',
    '<p class="MsoNormal">', '<p class="MsoNormal" style="line-height:normal">',
    '<p class="MsoNormal" style="margin-bottom:0.0001pt;line-height:normal"',
    '<p align="center" class="MsoNormal" style="text-align:center;line-height:normal">',
    ' style="font-size:11.5pt;letter-spacing:0.75pt" target="\_blank','\n\n>',
    '<a href="https://www.huffingtonpost.com/entry/trump-trade-war\\_us\\_5ac53929e4b09ef3b2432f64" title="這裏"></a>',
]
    
img_list = [
    ('f_26462958_1'), ('f_25350703_1'), ('2f50f0bffaf439ca5adac6dffafb4618'),
    ('ba77ae1763533d27063da5a3a6da38e0'), ('1255f7d9f3d7178b872880a1ffb96277'),
    ('1e42c210bafb4887d03c25669817fcd7'), ('14f1704e177d6c3985238567b5a613c0'),
    ('29e192984e9ce0086e88a98be3705562'), ('ab6f21134e7dcba1cfbf2e494200f9ef'),
    ('d39156c719714a726b82aae6048a3c41'), ('b314bf52394928e7e7fb213264129366'),
    ('d400eb09ec690963cfd6d651805ad4f0'), ('c624e9ae06a66f951f3d346c7698e9a3'),
]

#
content = """
\\documentclass[twocolumn]{ctexart}
\\usepackage{ctex}
\\usepackage{graphicx}
\\usepackage{titlesec}
\\usepackage[hyphens]{url}
\\usepackage[colorlinks=true, urlcolor=blue, linkcolor=black]{hyperref}
\\usepackage{geometry}
\\usepackage{cancel}
\\setCJKmainfont{SimSun}
\\setcounter{secnumdepth}{0}
\\setcounter{tocdepth}{1}
\\titleformat*{\\section}{\\centering\\Large\\bfseries }
\\titleformat*{\\subsection}{\\centering}
\\titlespacing*{\\subsection} {0pt}{0pt}{10ex}
\\geometry{a4paper, scale=0.85}
\\begin{document}
\\begin{titlepage}\\vspace*{8cm}\\begin{center}{\\Huge\\bfseries\\heiti 王孟源文集}(v23.6.14)\\end{center}\\end{titlepage}\\newpage
\\tableofcontents
\\newpage
\\pagestyle{plain}
"""
df_article = df_article.sort_values('art_date',ascending=True)
for idx in df_article.index:
    title = df_article.loc[idx,'title']
    date = df_article.loc[idx,'art_date'].strftime('%Y-%m-%d %H:%M')
    post = df_article.loc[idx,'post']
    if len(post) < 100:
        continue

    for rep in rep_list:
        post = post.replace(*rep)

    post = re.sub(r'<span .*?font-size.*?>','',post)
    post = re.sub(r'<span .*?font-family.*?>','',post)
    post = re.sub(r'<span style=.*?>','',post)
    post = re.sub(r'<div style=.*?>','',post)
    post = re.sub(r'<p style=.*?>','',post)
    post = re.sub(r'<font face=.*?>','',post)

    post = re.sub(f'[<span .*>]*<a\ +href="(.+?)".*?>http(.+?)<.*?\/a>[<\/span>]*',r'\\href{\1}{链接\\footnote{\\url{http\2}}}',post)
    post = re.sub(f'[<span .*>]*<a\ +href="(.+?)".*?>(.+?)<.*?\/a>[<\/span>]*',r'\\href{\1}{\2}',post)
    post = re.sub(f'<a href="(.*?)">',r'\\url{\1}',post)
    post = re.sub(f'<span .*標楷體.*?>(.*)<\/span>',r'{\\kaishu{\1}}',post)
    post = re.sub(r'<strong>(.*)<\/strong>',r'{\\centering\\heiti{\1}}',post)
    post = re.sub(r'<span .*?underline.*?>(.*)<\/span>',r'\\uline{\1}',post)

    for rem in rem_list:
        post = post.replace(rem,'')

    #image
    matches = re.findall('<img .*src="(.*)".*\/>',post)
    for match in matches:
        post = post.replace(match, match.replace('\_','_').replace('.gif','.jpg'))
        for img in img_list:
            post = post.replace(f'{img}.jpg',f'{img}.png')
    post = re.sub(f'<img .*src="(.*?)".*\/>',r'\\begin{figure}[h]\\centering\\includegraphics[width = 0.9\\linewidth]{../html/\1}\\end{figure}',post)
    post = post.replace('\\begin{figure}[h]\\centering\\includegraphics[width = 0.9\\linewidth]{../html/./img/empty.png}\\end{figure}','')
    post = post.replace('\\begin{figure}[h]\\centering\\includegraphics[width = 0.9\\linewidth]{../html/./img/empty.jpg}\\end{figure}','')

    #table
    if '<table' in post:
        loc1 = post.find('<table')
        loc2 = post.find('/table>')
        post = post.replace(post[loc1:loc2+7],'\\begin{figure}[h]\\centering\\includegraphics[width = 1\\linewidth]{../html/img/table.png}\\end{figure}')

    #list
    if '<ol>' in post:
        post = post.replace('<ol>','\\begin{enumerate}\n').replace('</ol>','\\end{enumerate}\n').replace('<li>','\\item ').replace('</li>','\n').replace('<p style="margin-left:30px">','\n')

    #special
    post = post.replace('P<sub>R</sub>(V)','$P^R(V)$')
    post = post.replace('</','').replace('a>','')
    
    content += f"\\twocolumn[\\begin{{@twocolumnfalse}}\n\\section{{{title}}}\n\\subsection{{{date}}}\n\\end{{@twocolumnfalse}}]"
    content += post

content += '\n \\end{document}'

with open('article.tex','w',encoding='utf8') as f:
    f.write(content)

In [4]:
import requests,os
import pandas as pd
from bs4 import BeautifulSoup

os.environ['https_proxy'] = '127.0.0.1:7890'
os.environ['http_proxy'] = '127.0.0.1:7890'

url = 'https://drive.google.com/drive/folders/1eg78LVciM913PhvRtsgtWV3VDLojynDA'
doc = BeautifulSoup(requests.get(url).content, features="lxml")
df = pd.DataFrame()
for i in doc.findAll("div", {'data-target':"doc"}):
    if i.find("div",{'role':"link"}):
        i.find("div",{'role':"link"}).decompose()
    df = pd.concat([df, pd.DataFrame(data={'filename': i.text, 'id': i['data-id']}, index=[0])], ignore_index=True)  
# L = tasker.run([saveScript(df.loc[idx, 'filename'][:6], df.loc[idx, 'id'], proxy) for idx in df.index])
# print('transcript downloaded')

In [5]:
df

,filename,id
0,wmyblog-ebook上午10:14—,1eLq7tAwzfFbfsQbgb4nA7WzRbaSCLOxR
1,170906[八方论坛]霸权交替 民选体制 武统 台湾的未来 ✅2023年6月2日一七 KB,1xYCICy678fw8ZVqunqXIuVyuTNTF6-_TKYmCO3TfoKM
2,170927[八方论坛]石油人民币✅2023年5月25日一八 KB,19He10g4o3_dugJErORx2Q3CQmJWFPy3Dwsl2-KI3SP4
3,171004[八方论坛]赌城枪击 比特币✅2023年5月14日二〇 KB,1J25_dpJ8hR1l0SbyHR2dAPRSt_lCWyW8T3I_hc6Hw-U
4,171115[八方论坛]哥本哈根诠释 量子纠缠 EPR 量子通信 密码学 知识分子✅2023...,1380MnCAmHp656Q2vEUd_tEpuPniLrdru1V1BVLShmNA
5,180420[八方论坛]中兴✅2023年5月14日二三 KB,1YQMEdCBeMyX7Lmzjxe9Yn2ZD4QPr9TEtD-76szqNOSg
6,180809[八方论坛]中美国力 贸易战交锋 智库 群众和民主制度 产业升级 期中选举✅20...,1Io1KSxkuYwx3P9EOyWdzn_zhRmyLdS5ub3psNfysSxQ
7,190129[八方论坛]华为✅2023年5月11日二五 KB,1USWPxYaXBbyGPqHhx753gm0YXUhdH8ilA5CPI76hQ_o
8,190512[八方论坛]中美贸易战✅2023年6月13日二〇 KB,1Hv7riKWjbTYUqXqOnvAHqyfe3FfLQs4ovQ33c282n8o
9,190812[八方论坛]香港动乱✅ 2023年5月11日二六 KB,1KB2SLafM5AAp8KYSN-Q7FkfJclIlumsCwpG86uDzvDo


In [3]:
'788bfa5c-a744-4dac-a22f-d3a6a8f0e64a'

'230601'

In [9]:
"\u8ba9\u5b50\u5f39\u98de.2010.BluRay.1080p.x264.DTS-HD.MA.5.1-HDWinG.mkv"

'让子弹飞.2010.BluRay.1080p.x264.DTS-HD.MA.5.1-HDWinG.mkv'

In [10]:
{"\u8ba9\u5b50\u5f39\u98de.2010.BluRay.1080p.x264.DTS-HD.MA.5.1-HDWinG.mkv": "https://du.smartont.net:8445/openapi/download?fid=54mJB_dgp5MvC0sXuDPMZn7buNzAByMXESUlzrBmAmbPUJORkB9SIJ5xToRLGKj06OTGBdABE5O3PyQYz4gMseQu/Ptw=="}["\u8ba9\u5b50\u5f39\u98de.2010.BluRay.1080p.x264.DTS-HD.MA.5.1-HDWinG.mkv"]

'https://du.smartont.net:8445/openapi/download?fid=54mJB_dgp5MvC0sXuDPMZn7buNzAByMXESUlzrBmAmbPUJORkB9SIJ5xToRLGKj06OTGBdABE5O3PyQYz4gMseQu/Ptw=='

In [2]:
from wmyblog_coron import table2tag

ImportError: attempted relative import with no known parent package

In [79]:
df0 = pd.read_excel('../data/table.xlsx')
df0 = df0.set_index(df0.columns[0],drop=True)
df0.to_csv('table.csv')

In [17]:
import requests,json
from bs4 import BeautifulSoup